In [26]:
#CRAWLING DATA FROM YAHOO MOVIES
import requests
import re
from bs4 import BeautifulSoup

Y_MOVIE_URL = "https://movies.yahoo.com.tw/movieinfo_main/"

# YOUR CODE HERE! 
# IMPLEMENTIG YAHOO MOVIES CRAWLER
class MovieCrawler(object):

    def __init__(self):
        self.session = requests.Session()
        
    def get_movies(self, page_url):
        movies = []
          # with open(page_url,'r') as f:
          #   contents = f.read()
        for i in range(2,3):
            response = self.session.get(page_url+str(i))#+"?page="+str(i))
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            movie_list = soup.find_all('div', id='content_l')
            # print(movie_list)
            for movie in movie_list:
                label_class = []
                # print(movie)
                try:
                    ch_name = movie.find('div', class_='movie_intro_info_r').h1#.text.strip()
                    try:
                        ch_name.span.decompose()
                    except:
                        pass
                    ch_name = ch_name.text.strip()
                    en_name = movie.find('div', class_='movie_intro_info_r').h3.text.strip()
                    release_date = movie.find('div', class_='movie_intro_info_r').find('span',class_=None).text[5:].strip()#.span.text.split('上映日期：')[-1].strip()
                    labels = movie.find_all('div',class_ = 'level_name')
                    for label in labels:
                        try:
                            lb = label.find('a',class_='gabtn').text.strip()
                        except:
                            break
                        label_class.append(lb)
                    intro = movie.find('div', class_='gray_infobox_inner').span.text.strip().replace('\n','')
                    doc_id = i
                    movie_info = {
                                'doc_id' : doc_id,
                                'ch_name': ch_name,
                                'en_name': en_name,
                                'label[class]': label_class,
                                'release_date': release_date,
                                'intro': intro}
                    movies.append(movie_info)
                except:
                    print(i)
                    continue
        return movies


# # DO NOT MODIFY THE VARIABLES
crawler = MovieCrawler()
movie = crawler.get_movies(Y_MOVIE_URL)

# # THE RESULTS : AS THE FOLLOWING SECTION
# # {'ch_name', 'en_name', 'movie_url', 'release_date', 'intro'}
print(len(movie))
print(*movie, sep="\n")
#SAVING DATA INTO JSON FILE

1
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [128]:
#SAVING DATA INTO JSON FILE
import json
with open("./moviedata.json", "w",encoding = 'utf-8') as final:
    json.dump(listObj, final,ensure_ascii=False)

In [131]:
#APPENDING NEW DATA INTO FILE
import json
with open("./moviedata.json", "r",encoding='utf-8') as fp:
    listObj = json.load(fp)
#     for m in movie:
#         listObj.append(m)
print(len(listObj))
print(listObj[1])
# with open("./hw2_1.json", "w",encoding = 'utf-8') as final:
#     json.dump(intro_all, final,ensure_ascii=False)

12238
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [111]:
import json
with open("./moviedata.json", "r",encoding = 'utf-8') as final:
    listObj = json.load(final)

In [130]:
for l in listObj:
    if 2 in l.values():
        print("true")
        print(l)

true
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [133]:
#INVERTED INDEX
import jieba
import re
from zhon.hanzi import punctuation
stopwords = [line.strip().replace('\n','') for line in open('../hw1/stopwords.txt','r').readlines()]
punctuations = [line.strip().replace('\n','') for line in open('../hw1/punctuation.txt','r').readlines()]
jieba.load_userdict('../hw1/userDict.txt')
punctuation = list(punctuation)

In [134]:
intro_all = []
for content in listObj:
    intro = content['intro'].replace('\n','').replace('\t','').replace('\u3000','')
    intro = re.sub('[^\u4e00-\u9fa5]','',intro)
    intro = jieba.lcut(intro)
    Intro = []
    for word in intro:
        if word not in stopwords:
            Intro.append(word)
    Intro.insert(0,content['ch_name'])
    Intro.insert(0,content['doc_id'])
    intro_all.append(Intro)

In [136]:
words = {}
for intro in intro_all:
    for word in intro:
        if word in words:
            words[word].append(intro[0])
        else:
            words[word] = [intro[0]]

In [ ]:
from collections import Counter
inverted_index = {}
for i,j in words.items():
    print(i)
    wordcount = (Counter(j))
    list(wordcount.items()).sort(key = lambda x : x[1],reverse = True)
    wordcounts = list(wordcount.items())
    inverted_index[i] = wordcounts

In [141]:
#PAGERANK
import networkx as nx
G = nx.DiGraph()
Edges = []
for word,doc_id in inverted_index.items():
    edges = []
    for ID in doc_id[2:]:
        if ID[0] != doc_id[0][0]:
            edges.append((doc_id[0][0],ID[0]))
        Edges.append(edges)

In [143]:
for edges in Edges:
    for edge in edges:
        G.add_edge(edge[0],edge[1])

In [ ]:
import networkx as nx
pagerank_list = nx.pagerank(G,alpha = 1)
print(pagerank_list)

In [ ]:
PageRankList = list(pagerank_list.items())
PageRankList.sort(key = lambda x:x[0])

In [233]:
#CREATING DOC LINKS
DOC_LINK = {}
doc_link = []
i = 0
for data in inverted_index.values():
    id_num = intro_all[i][0] 
    doc_id = data[0][0]
    for ID in data:
        if ID[0] not in doc_link and ID[0]!= doc_id:
            doc_link.append(ID[0])
    if doc_id != id_num:
        i+=1
        doc_link.sort()
        DOC_LINK[id_num] = doc_link
        doc_link = []
    

In [298]:
#SAVING DATA into json file hw2.json
# print(dict(listObj[0]))
DATA = []
for listobj in listObj:
    try:
        d_link = DOC_LINK[dict(listobj)['doc_id']]
    except:
        d_link = "None"
    try:
        prl = dict(PageRankList)[dict(listobj)['doc_id']]
    except:
        prl = 0
    info = {
        'doc_id' : dict(listobj)['doc_id'],
        'ch_name' : dict(listobj)['ch_name'],
        'en_name' : dict(listobj)['en_name'],
        'pagerank' : prl,
        'label[class]' : dict(listobj)['label[class]'],
        'intro' : dict(listobj)['intro'],
        'release_data' : dict(listobj)['release_date'],
        'links(doc_id)' : d_link,
    }
#     print(dict(listobj)['doc_id'])
    DATA.append(info)


In [302]:
#turn data into dictionary with id as key
movie_data = {}
for movie in DATA:
    movie_data[dict(movie)['doc_id']] = movie

# QUERY

In [440]:
class search_engine():
    def query(text):
        count_all = 0
        count = 0
        movie_prl = []
        movie_list = []
        #find all movie related to text
        regex = re.compile(rf'(.*){text}')
        for movie,data in movie_data.items():
            if regex.match(dict(data)['ch_name']) or regex.match(dict(data)['en_name']) or regex.match(dict(data)['intro']):
                count_all += 1
        for movie,index in inverted_index.items():
            if str(text) in str(movie):
                for i in index:
                    movie_list.append(i[0])
        #print all movies related with text
        movies_list=list(dict.fromkeys(movie_list))
        data = []
        for ID in movies_list:
            count+=1
            data.append([movie_data[ID]['doc_id'],movie_data[ID]['pagerank'],movie_data[ID]['ch_name'], movie_data[ID]['en_name'],movie_data[ID]['intro']])
        data.sort(key = lambda x:x[1],reverse = False)
        for movie in data:
            print("DOC_ID:",movie[0])
            print("CHINESE TITLE:",movie[2])
            print("ENGLISH TITLE:",movie[3])
            print("PAGERANK:",movie[1])
            print("INTRO:",movie[4])
            print("\n")
        print("Precision = "+ str(count/len(movie_list)*100)+"%")
        print("Recall = "+str(count/count_all*100)+"%")
        print(count)
        print(count_all)
        print(len(movie_list))
        

In [441]:
search_engine.query("捍衛戰士")

DOC_ID: 5248
CHINESE TITLE: 殲滅天際線
ENGLISH TITLE: Red Sky
PAGERANK: 6.119591928664552e-05
INTRO: ★《萬惡城市》製作群再現磅礡空戰，誓死炸翻大銀幕！麥斯是名飛行員，接獲上級任務進行殲滅行動，卻不料執行失誤，造成飛行戰隊極大損傷，麥斯也遭強迫除役。許久後，恐怖組織伺機而動，麥斯又被重招回部隊執行機密任務，卻發現他的朋友竟從昔日戰友變成敵軍。麥斯駕駛著戰鬥機，肩負著國家存亡，更為了平反個人榮辱，在浩瀚天際展開一場榮耀血戰。繼《捍衛戰士》、《空戰英豪》後，大銀幕上已經許久沒有出現空戰電影。此次由《萬惡城市》製片群再次重拾題材，斥資三億打造史詩級空戰。片中更結合當今反恐題材，讓故事更為貼近大眾。男主角肯吉甘特走紅於《暮光之城》，以《舞孃俱樂部》再次讓觀眾注意到這位帥氣小生。出眾的外型，被認為是布萊德彼特和柯林法瑞爾的合體，受到電影界的高度矚目。但31歲的他，其實十分專情，2009年與女友生下一女。揹著女兒逛大街的畫面，時常躍上雜誌報導。


DOC_ID: 14847
CHINESE TITLE: 噩夢輓歌 4K數位修復版
ENGLISH TITLE: Requiem For A Dream
PAGERANK: 7.295333251493753e-05
寡婦莎拉（艾倫鮑絲汀 飾）夢想穿著年輕時的紅洋裝上電視，於是決定靠藥物來瘦身。與此同時，她的兒子哈利（傑瑞德雷托 飾）和朋友泰隆（馬龍韋恩斯 飾）希望透過販售海洛因致富，哈利的女友瑪麗安（珍妮佛康納莉 飾）也加入他們的計畫。這時四人還不知道，藥物將使他們的人生走向不可挽回的境地……。


DOC_ID: 13382
CHINESE TITLE: 蜘蛛頭監獄
ENGLISH TITLE: Spiderhead
PAGERANK: 8.060887200117889e-05
有時候，藥物能讓他們感覺良好。想讓心情好起來嗎？不如試試這款藥。覺得無言以對？我們也有好藥可用。然而，同為受試者的傑夫 (Jeff)（麥爾斯·泰勒 (Miles Teller) 飾）與莉琪 (Lizzy)（朱妮·絲莫利特 (Jurnee Smollett) 飾）結為好友，兩人的救贖之路也隨著阿奈斯提的實驗將自由意志逼到極限，而出現更詭譎的轉折。結合多種戲劇類型，同